In [1]:
import os
os.chdir("../..")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix
import optuna
import os

C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

In [5]:
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(
    "wildfire_dataset_scaled", batch_size=32, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")

[INFO] Loading datasets...
[INFO] Datasets loaded successfully!


In [6]:
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])
    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    print("[DEBUG] Initializing mobilenet_v2 model...")
    model = initialize_model("mobilenet_v2", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
    model.to(device)

    # Optimizer setup
    print("[DEBUG] Setting up optimizer...")
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = "outputs/models/mobilenet_v2_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=5,
        monitor_metric="val_recall",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_recall"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation Recall: {val_recall:.4f}")
    return val_recall


In [7]:
# Run Optuna
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters found: {study.best_params}")

[I 2024-12-18 18:56:56,860] A new study created in memory with name: no-name-8cf8f440-3eef-4f42-8398-207c388e5ca1


[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[DEBUG] Suggested hyperparameters: lr=0.002060405581092832, weight_decay=6.102976271899184e-05, optimizer=Adam
[DEBUG] Initializing mobilenet_v2 model...


C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[DEBUG] Setting up optimizer...
[INFO] Starting model training...

Starting training...



Epoch [1]: Train Loss: 0.5522, Train Acc: 0.7515 | Val Loss: 0.4618, Val Acc: 0.7736, Val Recall: 0.9024, Val F1: 0.8299
[INFO] Best model saved with val_recall: 0.9024
[INFO] Learning rate adjusted to: 0.002060


...

Epoch [9]: Train Loss: 0.4082, Train Acc: 0.8241 | Val Loss: 0.4475, Val Acc: 0.7910, Val Recall: 0.8943, Val F1: 0.8397
[INFO] No improvement in val_recall. Patience: 2/5
[INFO] Learning rate adjusted to: 0.004255


Epoch [10]: Train Loss: 0.3661, Train Acc: 0.8453 | Val Loss: 0.4243, Val Acc: 0.7836, Val Recall: 0.9065, Val F1: 0.8368
[INFO] No improvement in val_recall. Patience: 3/5
[INFO] Learning rate adjusted to: 0.004255


Epoch [11]: Train Loss: 0.3806, Train Acc: 0.8352 | Val Loss: 0.3878, Val Acc: 0.8159, Val Recall: 0.9309, Val F1: 0.8609
[INFO] Best model saved with val_recall: 0.9309
[INFO] Learning rate adjusted to: 0.004255


[W 2024-12-18 19:11:44,512] Trial 7 failed with parameters: {'lr': 0.004255450414076736, 'weight_decay': 0.0006437536330638324, 'optimizer': 'AdamW'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Yash\AppData\Local\Temp\ipykernel_10436\3609394824.py", line 33, in objective
    history = train_model(
              ^^^^^^^^^^^^
  File "C:\Users\Yash\Documents\Development\Projects\final_year\utils\train_utils.py", line 68, in train_model
    train_loss += loss.item() * images.size(0)
                  ^^^^^^^^^^^
KeyboardInterrupt
[W 2024-12-18 19:11:44,538] Trial 7 failed with value None.


KeyboardInterrupt: 

In [ ]:
# Train Final Model
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model("mobilenet_v2", num_classes=2, pretrained=True, freeze_all=False, unfreeze_last_n=2)
final_model.to(device)

In [ ]:
# Optimizer
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params["lr"], weight_decay=study.best_params["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

In [ ]:
# Final training
final_save_path = "outputs/models/mobilenet_v2_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=nn.CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=5,
    monitor_metric="val_recall",
)

In [ ]:
# Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)
plot_training(history, "outputs/plots/mobilenet_v2_training_curve.png")
plot_confusion_matrix(metrics["confusion_matrix"], ["No Fire", "Fire"], "outputs/plots/mobilenet_v2_confusion_matrix.png")
print("[INFO] Final results saved successfully!")